# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [ ]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.

In [2]:
function mul(A, x)
    m, n = size(A)
    ret = zeros(eltype(A), m)
    for j = 1:n
        for i = 1:m
            ret[i] += A[i, j] * x[j]
        end
    end
    ret
end;

In [4]:
n = 10
A = randn(n, n)
x = randn(n)

A*x == mul(A, x)

false

## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [5]:
function ldiv(A::UpperTriangular, b)
    n = size(A,1)
    x = zeros(n)
    
    if length(b) != n
        error("The system is not compatible")
    end
    
    for k = n:-1:1
        x[k] = (b[k] - sum([A[k, j]*x[j] for j = k+1:n]))/A[k, k]
    end
    x
end;

function ldiv(A::LowerTriangular, b)
    n = size(A,1)
    x = zeros(n)
    
    if length(b) != n
        error("The system is not compatible")
    end
    
    for k = 1:n
        x[k] = (b[k] - sum([A[k, j]*x[j] for j = 1:k-1]))/A[k, k]
    end
    x
end;

In [6]:
# Test matrix
n = 10
A = randn(n, n)
x = randn(n)

U = UpperTriangular(A)
L = LowerTriangular(A)

b_U = U*x
b_L = L*x;

In [7]:
@test ldiv(U, b_U) ≈ x && ldiv(L, b_L) ≈ x

Test Passed
  Expression: ldiv(U, b_U) ≈ x && ldiv(L, b_L) ≈ x

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [8]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end;

size(U::UpperTridiagonal) = (length(U.d),length(U.d));

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    diags = [U.d,U.du,U.du2]

    if k <= j && j <= k + 2
        ret = diags[j - k + 1][k]
    else
        ret = 0
    end
    ret
end;

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    diags = [U.d,U.du,U.du2]
    if j > k+2
        error("Cannot modify off-band")
    end

    diags[j - k][k] = v
    U
end;

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [9]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    n = size(U,1)

    [sum([U[i, j]*x[j] for j = i:min(i+2, n)]) for i = (1:n)]
end;

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    n = size(U,1)
    x = zeros(T, size(U,2))

    for k = n:-1:1
        x[k] = (b[k] - sum([U[k, j]*x[j] for j = k+1:min(k+3, n)]))/U[k, k]
    end
    x
end;

In [10]:
# Test matrix
n = 10

d = randn(n)
du = randn(n-1)
du2 = randn(n-2)
diags = [d, du, du2]

U = UpperTridiagonal(d, du, du2)
x = randn(n)

A = zeros(n,n)
for i = 1:n
    for j = 1:min(3, n-i+1)
        A[i, j+i-1] = diags[j][i]
    end
end

In [11]:
@test A*x ≈ U*x && U\(U*x) ≈ x

Test Passed
  Expression: A * x ≈ U * x && U \ (U * x) ≈ x

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [12]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    Int(j == P.p[k])
end

function *(P::PermutationMatrix, x::AbstractVector)
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$


**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.


**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [13]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    ==(k, j) - 2*Q.v[k]*Q.v[j]
end

function *(P::Reflection, x::AbstractVector)
    (I - 2P.v*P.v')*x
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0 -2.6020852139652106e-18 … 1.9081958235744878e-17 -3.469446951953614e-18; -2.6020852139652106e-18 1.0 … 1.5265566588595902e-16 -5.551115123125783e-17; … ; 1.9081958235744878e-17 1.5265566588595902e-16 … 1.0000000000000002 -1.1102230246251565e-16; -3.469446951953614e-18 -5.551115123125783e-17 … -1.1102230246251565e-16 1.0] ≈ UniformScaling{Bool}(true)

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass. Here `s == true` means the Householder reflection is sent to the positive axis and `s == false` is the negative axis.

In [14]:
function householderreflection(s::Bool, x::AbstractVector)
    y = copy(x)
    y[1] += -(2s - 1)*norm(x)
    w = y/norm(y)

    Reflection(w)
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ [norm(x);zeros(eltype(x),length(x)-1)]

Q = householderreflection(false, x)
@test Q isa Reflection
@test Q*x ≈ [-norm(x);zeros(eltype(x),length(x)-1)]

Test Passed
  Expression: Q * x ≈ [-(norm(x)); zeros(eltype(x), length(x) - 1)]
   Evaluated: [-2.0381511574659714, 2.6020852139652106e-17, 1.8388068845354155e-16, -3.469446951953614e-16, 1.2917987639212584e-17] ≈ [-2.038151157465971, 0.0, 0.0, 0.0, 0.0]